In [1]:
# First cell - Setup paths
import sys
import os

# Setup paths as before
notebook_dir = os.path.dirname('__file__')
lerobot_root = os.path.abspath(os.path.join(notebook_dir, '../..'))
sys.path.append(lerobot_root)
os.chdir(lerobot_root)

In [2]:
import re
import cv2
import matplotlib.pyplot as plt
import time
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import json
import PIL
from pathlib import Path
import google.generativeai as genai

from dataclasses import dataclass
from pathlib import Path
from lerobot.common.robot_devices.control_configs import RecordControlConfig
from lerobot.scripts.control_robot import record
from lerobot.common.robot_devices.robots.utils import make_robot_from_config
from lerobot.common.robot_devices.control_utils import init_keyboard_listener
from lerobot.common.robot_devices.robots.configs import So100RobotConfig
from lerobot.notebooks.robocontrol import RobotController
from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig
from lerobot.common.robot_devices.control_utils import busy_wait
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

In [4]:
class KinesicsRecorder:
    def __init__(self, cameras, save_dir="./kinesics_data"):
        """Initialize recorder with robot config and save directory"""
        # Create save directory
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        
        # Configure robot with existing camera config
        self.robot_config = So100RobotConfig(
            cameras=cameras,
            mock=False
        )
        
        # Initialize robot
        self.robot = make_robot_from_config(self.robot_config)
        
    def record_session(self, session_name, task_description, duration=30):
        """Record a teleoperation session
        
        Args:
            session_name (str): Name for this recording session
            task_description (str): Description of the task being recorded
            duration (int): Recording duration in seconds
        """
        try:
            # Create unique session directory
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            unique_name = f"{session_name}_{timestamp}"
            session_dir = self.save_dir / unique_name
            
            # Make sure directory doesn't exist
            if session_dir.exists():
                print(f"Warning: {session_dir} already exists, using new timestamp")
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                unique_name = f"{session_name}_{timestamp}"
                session_dir = self.save_dir / unique_name
            
            print(f"Starting recording session: {unique_name}")
            print(f"Task: {task_description}")
            print("Press right arrow to stop early, ESC to exit")
            
            # Initialize keyboard listener
            listener, events = init_keyboard_listener()
            
            # Create config for recording
            cfg = RecordControlConfig(
                fps=30,
                episode_time_s=duration,
                display_cameras=False,
                repo_id=f"kinesics/{unique_name}",
                num_episodes=1,
                warmup_time_s=2,
                reset_time_s=0,
                root=str(session_dir),
                push_to_hub=False,
                video=True,
                run_compute_stats=False,
                single_task=task_description
            )
            
            # Record using existing function
            dataset = record(
                robot=self.robot,
                cfg=cfg
            )
            
            print(f"Recording complete. Saved to {session_dir}")
            return dataset
            
        except Exception as e:
            print(f"Error recording session: {e}")
            raise
        finally:
            if self.robot.is_connected:
                self.robot.disconnect()
    
    def extract_action(self, session_name, action_name, start_time, end_time):
        """Extract an action sequence from a recorded session"""
        try:
            # Find session directory code remains same...
            session_dirs = list(self.save_dir.glob(f"{session_name}*"))
            if not session_dirs:
                raise ValueError(f"No recorded session found with name: {session_name}")
            
            session_dir = max(session_dirs, key=lambda p: p.stat().st_mtime)
            
            # Load dataset
            dataset = LeRobotDataset(
                repo_id=f"kinesics/{session_dir.name}",
                root=str(session_dir),
                local_files_only=True
            )
            
            # Calculate frame indices
            fps = dataset.fps
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)
            
            if start_frame < 0 or end_frame >= dataset.num_frames:
                raise ValueError(f"Invalid time range. Session length: {dataset.num_frames/fps:.2f}s")
            
            # Get frames and extract actions
            frames = dataset.hf_dataset[start_frame:end_frame]
            actions = frames['action']
            
            # Convert list of tensors to numpy array
            joint_positions = np.array([tensor.numpy() for tensor in actions])
            
            # Save extracted action
            action_dir = self.save_dir / "actions"
            action_dir.mkdir(exist_ok=True)
            
            action_data = {
                "name": action_name,
                "joint_positions": joint_positions,
                "fps": fps,
                "metadata": {
                    "source_session": session_dir.name,
                    "start_time": start_time,
                    "end_time": end_time,
                    "extracted_at": datetime.now().isoformat(),
                    "joint_names": dataset.features["action"]["names"]  # Save joint names
                }
            }
            
            np.savez(
                action_dir / f"{action_name}.npz",
                **action_data
            )
            
            print(f"\nExtracted action '{action_name}' ({end_time-start_time:.2f}s)")
            print(f"Joint positions shape: {joint_positions.shape}")
            return action_data
            
        except Exception as e:
            print(f"Error extracting action: {e}")
            raise
            
    def replay_action(self, action_name, transition_time=1.0):
        """Replay a saved action with smooth transition
        
        Args:
            action_name (str): Name of the action to replay
            transition_time (float): Time in seconds for smooth transition
        """
        try:
            # Connect robot if not connected
            if not self.robot.is_connected:
                print("Connecting robot...")
                self.robot.connect()
            
            # Load action data
            action_path = self.save_dir / "actions" / f"{action_name}.npz"
            if not action_path.exists():
                raise ValueError(f"Action not found: {action_name}")
                
            action_data = np.load(action_path, allow_pickle=True)
            joint_positions = action_data["joint_positions"]
            fps = float(action_data["fps"])
            
            # Get current robot state
            observation = self.robot.capture_observation()
            current_state = observation["observation.state"]  # Changed from ["state"]
            
            print(f"Current state shape: {current_state.shape}")
            print(f"Target state shape: {joint_positions[0].shape}")
            
            # Calculate transition trajectory
            transition_steps = int(transition_time * fps)
            start_pos = current_state
            end_pos = joint_positions[0]
            
            transition = np.linspace(start_pos, end_pos, transition_steps)
            
            # Execute transition
            print("Starting transition...")
            for pos in transition:
                self.robot.send_action(torch.from_numpy(pos))
                busy_wait(1.0/fps)
                
            # Execute action
            print(f"Replaying action: {action_name}")
            for pos in joint_positions:
                self.robot.send_action(torch.from_numpy(pos))
                busy_wait(1.0/fps)
                
            print("Replay complete")
            
        except Exception as e:
            print(f"Error replaying action: {e}")
            print("Observation keys:", observation.keys())  # Debug info
            raise
        finally:
            if self.robot.is_connected:
                print("Disconnecting robot...")
                self.robot.disconnect()

In [5]:
# Create camera config using proper config objects
# Create camera config using proper config objects
cameras = {
    "laptop": OpenCVCameraConfig(
        camera_index=0,  # Built-in webcam
        fps=30,
        width=640,
        height=480
    ),
    # "phone": OpenCVCameraConfig(
    #     camera_index=1,  # iPhone camera
    #     fps=30,
    #     width=640,
    #     height=480
    # )
}

In [6]:
# Test recording
recorder = KinesicsRecorder(cameras=cameras, save_dir="/Users/shreyas/Downloads/trex/kinesics")


In [ ]:
dataset = recorder.record_session(
    session_name="reminder_to_focus",
    task_description="reminder_to_focus",
    duration=20
)

In [ ]:
# Extract an action from a recorded session
recorder.extract_action(
    session_name="reminder_to_focus",
    action_name="reminder_to_focus",
    start_time=5,  # Start 2 seconds in
    end_time=11  # End at 5 seconds
)

In [ ]:
# Replay the extracted action
recorder.replay_action("nod", transition_time=1.0)